In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import oct2py
from matpower import start_instance

In [3]:
oct2py.__version__

'5.7.2'

In [4]:
m = start_instance()

In [5]:
mpc = m.loadcase("case9", verbose=False)
mpc

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1. ,   3. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  2. ,   2. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  3. ,   2. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  4. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  5. ,   1. ,  90. ,  30. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  6. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  7. ,   1. , 100. ,  35. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  8. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
         345. ,   1. ,   1.1,   0.9],
        [  9. ,   1. , 125. ,  50. ,   0. ,   0. ,   1. ,   1. ,   0. ,
     

In [6]:
# foo = "save: wrong type argument 'function handle'"
# NOTE: bug on 'octave::execution_exception'
# NOTE: bug submitted to https://savannah.gnu.org/bugs/index.php?66060
m.eval("warning('off', 'Octave:classdef-to-struct');")
m.eval("warning('off', 'oct2py:pyeval:save_safe_struct:UnacceptableType');")
mpopt = m.mpoption("out.all", 0)
_ = m.runpf(mpc, mpopt, verbose=True)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful


In [7]:
m.push("_mpc", mpc, verbose=False)
m.eval("_r1 = runpf(_mpc);", verbose=True)
fieldnames_r1 = m.eval("fieldnames(_r1);", verbose=False)
fieldnames_r1.ravel()


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.09 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers    

Cell(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'order',
       'task', 'et', 'success', 'iterations', 'om'])

In [8]:
for i in fieldnames_r1.ravel():
    # print('start')
    # print(i)
    try:
        val = m.eval(f"_r1.{i};", verbose=True)
        # print(type(val))
        # print(val)
    except Exception as e:
        print(i)
        print(e)
        var_type = m.eval(f"class(_r1.{i});", verbose=False)
        print(var_type)
    # print('end')

    _pyeval>save_safe_struct at line 131 column 9
    _pyeval at line 97 column 3

task
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 131, column 9
mp.task_pf_legacy
    _pyeval>save_safe_struct at line 131 column 9
    _pyeval at line 97 column 3

om
Octave evaluation error:
error: save: error while writing '' to MAT file
error: called from:
    _pyeval>save_safe_struct at line 131, column 9
mp.math_model_pf_acps


In [9]:
for i in ["om"]:
    fieldnames_om = m.eval(f"fieldnames(_r1.{i});", verbose=False)
    print(fieldnames_om.ravel())

['aux_data' 'element_classes' 'elements' 'lin' 'nlc' 'nle' 'nli'
 'prob_type' 'qdc' 'set_types' 'soln' 'userdata' 'var']


In [10]:
for i in fieldnames_om.ravel():
    # print('start')
    # print(i)
    try:
        val = m.eval(f"_r1.om.{i};", verbose=False)
        # print(type(val))
        # print(val)
    except Exception as e:
        print("start")
        print(i)
        print(e)
        var_type = m.eval(f"class(_r1.om.{i});", verbose=False)
        print(var_type)
        print("end")
        pass
    # print('end')

start
elements
'NoneType' object is not iterable
mp.mapped_array
end
start
nle
Octave evaluation error:
error: invalid dot name structure assignment because the structure array is empty.  Specify a subscript on the structure array to resolve.
error: called from:
    _pyeval>clean_struct at line 150, column 36
    _pyeval>clean_struct at line 148, column 36
    _pyeval>save_safe_struct at line 127, column 25
struct
end


In [11]:
m.push("_mpc", mpc)
m.eval("_r1 = runopf(_mpc);", verbose=False)
m.eval("_r1.raw = rmfield(_r1.raw, 'task');")
m.eval("_r1 = rmfield(_r1, 'om');")
# m.eval("struct_levels_to_print(3)")
# m.eval("_r1")
sol = m.pull("_r1")
sol

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[ 1.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.09995086e+00,  0.00000000e+00,
          3.45000000e+02,  1.00000000e+00,  1.10000000e+00,
          9.00000000e-01,  2.47556946e+01,  2.16496816e-08,
          8.38437649e+00,  0.00000000e+00],
        [ 2.00000000e+00,  2.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.09736288e+00,  4.89310902e+00,
          3.45000000e+02,  1.00000000e+00,  1.10000000e+00,
          9.00000000e-01,  2.40345108e+01, -1.14060074e-09,
          0.00000000e+00,  0.00000000e+00],
        [ 3.00000000e+00,  2.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.08662735e+00,  3.24900488e+00,
          3.45000000e+02,  1.00000000e+00,  1.10000000e+00,
          9.00000000e-01,  2.

In [12]:
from oct2py.core import Oct2PyError

try:
    m.runopf(mpc, verbose=True)
except Oct2PyError as e:
    print(e)


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
Converged!
OPF successful

Converged in 0.20 seconds
Objective Function Value = 5296.69 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    318.3              -9.6
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)          

In [13]:
m.exit()